In [15]:
import json
import ast
import datetime
import time

In [34]:
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

corupt_sub_counter = 0
c_test, c_train= 0, 0
corpus_test, corpus_train= [], []
start_epoch=int(datetime.datetime(2017, 9, 1).timestamp())
end_epoch=int(datetime.datetime(2018, 3, 30).timestamp())

with open("bitcoin_cascades_subs_from_dic.txt", "r") as f,  open("corpus_train.txt", "w") as ctrain,  open("corpus_test.txt", "w") as ctest:
    for line in f:
        c_train += 1
        line_parts = line.split(';', 1)
        id = line_parts[0]
        submission = ast.literal_eval(line_parts[1])
        
        epoch, title, body = 0, "", ""
        try:
            epoch = int(submission['time'])
            title = submission['title']
            body = submission['body']

        except TypeError:
            corupt_sub_counter += 1
        
        corpa = title + ' ' + body + ' '
        corpus_train.append(corpa)
        ctrain.write(corpa + "\n")
        
        if epoch >= start_epoch and epoch <= end_epoch :
            c_test += 1
            corpus_test.append(corpa)
            ctest.write(corpa + "\n")
            
print("Train: ", len(corpus_train))
print("Test: ", len(corpus_test))
print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))


Starting at 2019-10-20 18:18:53.986400
Train:  206198
Test:  176442
Finished at 2019-10-20 18:19:10.462135
Total time: --- 16.4759 seconds ---


In [16]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [17]:
# put the common code into several methods
def get_keywords(crp):

    feature_names=cv.get_feature_names()
    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([crp]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("crypto_stopwords.txt")

In [ ]:
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(corpus_train)

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [39]:
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

keyword_dict = dict()

for cor in corpus_test:
    keywords=get_keywords(cor)
    for k in keywords:
        if k in keyword_dict:
            keyword_dict[k] += 1
        else:
            keyword_dict[k] = 1
            
print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))

Starting at 2019-10-20 18:19:18.244604
Finished at 2019-10-20 23:20:34.034372
Total time: --- 18075.7904 seconds ---


In [22]:
print(len(keyword_dict))

106


In [23]:
sorted_x = sorted(keyword_dict.items(), key=lambda kv: kv[1])

In [24]:
sorted_x[len(sorted_x) - 50: len(sorted_x)]

[('charge', 1),
 ('80k', 1),
 ('details', 1),
 ('30', 1),
 ('25', 1),
 ('2018', 1),
 ('exchange', 2),
 ('miss', 2),
 ('x200b', 2),
 ('last', 2),
 ('chance', 2),
 ('days', 2),
 ('results', 2),
 ('great', 2),
 ('week', 2),
 ('100k', 2),
 ('here', 2),
 ('scammers', 2),
 ('legit', 2),
 ('knows', 2),
 ('knowledgeable', 2),
 ('kind', 2),
 ('intensity', 2),
 ('new', 3),
 ('vkfctvb', 3),
 ('saturday', 3),
 ('invitation', 3),
 ('youtube', 3),
 ('www', 3),
 ('free', 3),
 ('invites', 3),
 ('binance', 3),
 ('now', 4),
 ('vip', 4),
 ('big', 4),
 ('members', 4),
 ('com', 4),
 ('recently', 4),
 ('limited', 4),
 ('achtjgw', 4),
 ('vibe', 5),
 ('wanna', 5),
 ('just', 5),
 ('invite', 6),
 ('check', 7),
 ('watch', 9),
 ('cvaja4x', 10),
 ('gg', 11),
 ('discord', 12),
 ('https', 12)]

In [47]:
sorted_x_dict = dict(sorted_x)

In [48]:
import json
with open("btc_sub_keywords.txt", 'w') as f:
    json_d = json.dumps(sorted_x_dict)
    f.write(json_d)
    f.close()

### Get cryptoPump subreddit keywords

In [2]:
import datetime as dt
import json
import requests 
import networkx as nx
import time
import datetime

In [4]:
comment_ids_endpoint_url = 'https://api.pushshift.io/reddit/submission/comment_ids/'
comment_endpoint_url = 'https://api.pushshift.io/reddit/comment/search/'
submission_endpoint_url = 'https://api.pushshift.io/reddit/submission/search/'

In [5]:
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

sub_counter = 0

submission_url = submission_endpoint_url + "?subreddit=" + 'cryptopump' + "&fields=selftext,title"

with open("cryptoPumpSubreddit.txt", "w") as f:
    
    sub_r = requests.get(url = submission_url)
    print("Submission URL:", submission_url)
    
    if sub_r.status_code == 200:
        sub_json_obj = json.loads(sub_r.text)
        sub_arr = sub_json_obj['data']

    for s in sub_arr:
        f.write(str(s) + '\n')
        sub_counter += 1
        
print(str(sub_counter), "submissions recieved")
print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))

Starting at 2019-11-13 10:31:46.387832
Submission URL: https://api.pushshift.io/reddit/submission/search/?subreddit=cryptopump&fields=selftext,title
25 submissions recieved
Finished at 2019-11-13 10:31:46.650533
Total time: --- 0.2629 seconds ---


In [12]:
#### Reading from the dictionary file we created
import ast

start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

corupt_sub_counter = 0
c_test, c_train= 0, 0
corpus_test, corpus_train= [], []
start_epoch=int(datetime.datetime(2017, 9, 1).timestamp())
end_epoch=int(datetime.datetime(2018, 3, 30).timestamp())

with open("cryptoPumpSubreddit.txt", "r") as f,  open("cryptoPumpSubreddit_corpus_train.txt", "w") as ctrain:
    for line in f:
        json_obj = ast.literal_eval(line)
        title = json_obj['title']
        body = json_obj['selftext']
        
        c_train += 1        
        corpa = title + ' ' + body + ' '
        corpus_train.append(corpa)
        ctrain.write(corpa + "\n")
            
print("Train: ", len(corpus_train))
print("Test: ", len(corpus_test))
print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))

Starting at 2019-11-13 11:01:14.539593
Train:  25
Test:  0
Finished at 2019-11-13 11:01:14.542890
Total time: --- 0.0035 seconds ---


In [19]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(corpus_train)

/home/pbashiri/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [20]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [21]:
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

keyword_dict = dict()

for cor in corpus_train:
    keywords=get_keywords(cor)
    for k in keywords:
        if k in keyword_dict:
            keyword_dict[k] += 1
        else:
            keyword_dict[k] = 1
            
print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))

Starting at 2019-11-13 11:04:12.199104
Finished at 2019-11-13 11:04:12.249935
Total time: --- 0.0514 seconds ---


In [25]:
sorted_x

[('participant', 1),
 ('only', 1),
 ('airdrops', 1),
 ('600', 1),
 ('tomorrow', 1),
 ('returns', 1),
 ('next', 1),
 ('expected', 1),
 ('announced', 1),
 ('2000', 1),
 ('zbsxprf', 1),
 ('back', 1),
 ('real', 1),
 ('oldest', 1),
 ('megapumpgroup', 1),
 ('mature', 1),
 ('don', 1),
 ('amp', 1),
 ('token', 1),
 ('profits', 1),
 ('pkg', 1),
 ('mercatox', 1),
 ('buy', 1),
 ('10000x', 1),
 ('100000x', 1),
 ('scheduled', 1),
 ('know', 1),
 ('first', 1),
 ('hour', 1),
 ('begins', 1),
 ('90000', 1),
 ('x887ju7', 1),
 ('today', 1),
 ('plus', 1),
 ('huuuuuge', 1),
 ('event', 1),
 ('come', 1),
 ('7pm', 1),
 ('90k', 1),
 ('learn', 1),
 ('youd', 1),
 ('wel', 1),
 ('xahafp7', 1),
 ('dont', 1),
 ('close', 1),
 ('pamp', 1),
 ('bigpumpsignal', 1),
 ('12d1kjww42', 1),
 ('day', 1),
 ('instructions', 1),
 ('biggest', 1),
 ('spot', 1),
 ('snag', 1),
 ('pumping', 1),
 ('member', 1),
 ('coin', 1),
 ('charge', 1),
 ('80k', 1),
 ('details', 1),
 ('30', 1),
 ('25', 1),
 ('2018', 1),
 ('exchange', 2),
 ('miss', 2),

In [50]:
train_corpus_str = "%%%%".join(str(x) for x in corpus_train)
train_corpus_str

"🌷📌AIRDROPS FOR 600 PARTICIPANT ONLY [removed] %%%%Next Pump Announced Tomorrow! Expected returns are 2000% (New Exchange!!) [removed] %%%%MEGAPUMPGROUP IS BACK!! the oldest and most mature and real pump group is now back! check it out now!!\n\n  \n\n[https://discord.gg/ZBSxPRF](https://discord.gg/ZBSxPRF)\n\n  \n\n[https://discord.gg/ZBSxPRF](https://discord.gg/ZBSxPRF)\n\n  \n\n[https://discord.gg/ZBSxPRF](https://discord.gg/ZBSxPRF)\n\n&amp;#x200B; %%%%This Saturday - another pump! Don't miss it VIP invite to join: [https://discord.gg/vKfCTVb](https://discord.gg/vKfCTVb)\n\n&amp;#x200B; %%%%PKG Token buy now on mercatox exchange 10000x to 100000x profits 💯%  %%%%New big pump scheduled! Be the first to know more about it! For more info, VIP invitation here: [https://discord.gg/vKfCTVb](https://discord.gg/vKfCTVb) %%%%In 1 hour, very big pump!!! 90000 + members.\n\nIn 1 hour the pump begins! \n\nLast chance to join here: [https://discord.gg/vKfCTVb](https://discord.gg/vKfCTVb)  %%%%HU

In [66]:
train_corpus_str

"🌷📌AIRDROPS FOR 600 PARTICIPANT ONLY [removed] %%%%Next Pump Announced Tomorrow! Expected returns are 2000% (New Exchange!!) [removed] %%%%MEGAPUMPGROUP IS BACK!! the oldest and most mature and real pump group is now back! check it out now!!\n\n  \n\n[https://discord.gg/ZBSxPRF](https://discord.gg/ZBSxPRF)\n\n  \n\n[https://discord.gg/ZBSxPRF](https://discord.gg/ZBSxPRF)\n\n  \n\n[https://discord.gg/ZBSxPRF](https://discord.gg/ZBSxPRF)\n\n&amp;#x200B; %%%%This Saturday - another pump! Don't miss it VIP invite to join: [https://discord.gg/vKfCTVb](https://discord.gg/vKfCTVb)\n\n&amp;#x200B; %%%%PKG Token buy now on mercatox exchange 10000x to 100000x profits 💯%  %%%%New big pump scheduled! Be the first to know more about it! For more info, VIP invitation here: [https://discord.gg/vKfCTVb](https://discord.gg/vKfCTVb) %%%%In 1 hour, very big pump!!! 90000 + members.\n\nIn 1 hour the pump begins! \n\nLast chance to join here: [https://discord.gg/vKfCTVb](https://discord.gg/vKfCTVb)  %%%%HU

In [87]:
regex = 'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/\w*(?:\?v=\w*)?-?\w*'
urls = re.findall(regex, train_corpus_str)
urls

['https://discord.gg/ZBSxPRF',
 'https://discord.gg/ZBSxPRF',
 'https://discord.gg/ZBSxPRF',
 'https://discord.gg/ZBSxPRF',
 'https://discord.gg/ZBSxPRF',
 'https://discord.gg/ZBSxPRF',
 'https://discord.gg/vKfCTVb',
 'https://discord.gg/vKfCTVb',
 'https://discord.gg/vKfCTVb',
 'https://discord.gg/vKfCTVb',
 'https://discord.gg/vKfCTVb',
 'https://discord.gg/vKfCTVb',
 'https://discord.gg/X887JU7',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://www.youtube.com/watch?v=9Ctq1N',
 'https://www.youtube.com/watch?v=9Ctq1N_Mrpk',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://www.youtube.com/watch?v=9Ctq1N',
 'https://www.youtube.com/watch?v=9Ctq1N_Mrpk',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/CVaJA4x',
 'https://www.youtube.com/watch?v=9Ctq1N',
 'https://www.youtube.com/watch

In [89]:
set(urls)

{'https://bigpumpsignal.com/invite-12d1kjww42',
 'https://discord.gg/CVaJA4x',
 'https://discord.gg/X887JU7',
 'https://discord.gg/ZBSxPRF',
 'https://discord.gg/achTjGW',
 'https://discord.gg/vKfCTVb',
 'https://discord.gg/xAhafp7',
 'https://www.youtube.com/watch?v=9Ctq1N',
 'https://www.youtube.com/watch?v=9Ctq1N_Mrpk'}

In [78]:
t = 'https://www.youtube.com/watch?v=9Ctq1N_Mrpk'
regex = 'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/\w*\??'
urls = re.findall(regex, t)
urls

['https://www.youtube.com/watch?']

In [88]:
len(urls)

67

In [55]:
import copy

train_corpus_str_no_url = copy.deepcopy(train_corpus_str)

for url in urls:
    train_corpus_str_no_url = train_corpus_str_no_url.replace(url, '')

In [57]:
train_corpus_ls_no_url = train_corpus_str_no_url.split('%%%%')

In [58]:
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

keyword_dict = dict()

for cor in train_corpus_ls_no_url:
    keywords=get_keywords(cor)
    for k in keywords:
        if k in keyword_dict:
            keyword_dict[k] += 1
        else:
            keyword_dict[k] = 1
            
print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))

Starting at 2019-11-13 11:37:32.769794
Finished at 2019-11-13 11:37:32.805482
Total time: --- 0.0359 seconds ---


In [60]:
sorted_x = sorted(keyword_dict.items(), key=lambda kv: kv[1])

In [65]:
len(sorted_x)

103

In [93]:
len(list([i for i in sorted_x if i[1] >2]))

25

In [98]:
keys_sorted = [k for k, v in sorted_x]

In [99]:
keys_sorted

['participant',
 'only',
 'airdrops',
 '600',
 'tomorrow',
 'returns',
 'next',
 'expected',
 'announced',
 '2000',
 'back',
 'real',
 'oldest',
 'megapumpgroup',
 'mature',
 'don',
 'token',
 'profits',
 'pkg',
 'mercatox',
 'buy',
 '10000x',
 '100000x',
 'scheduled',
 'know',
 'hour',
 'begins',
 '90000',
 'today',
 'plus',
 'huuuuuge',
 'event',
 'come',
 '7pm',
 '90k',
 'learn',
 'youd',
 'wel',
 'wants',
 'visit',
 'tech',
 'signals',
 'shows',
 'pumps',
 'like',
 'dont',
 'close',
 'pamp',
 '12d1kjww42',
 'day',
 'instructions',
 'spot',
 'snag',
 'pumping',
 'member',
 'coin',
 'charge',
 '80k',
 'details',
 '30',
 '25',
 '2018',
 'exchange',
 'miss',
 'last',
 'chance',
 'days',
 'results',
 'great',
 'biggest',
 'week',
 'scammers',
 'legit',
 'knows',
 'knowledgeable',
 'kind',
 'intensity',
 'high',
 'new',
 'amp',
 'saturday',
 'invitation',
 'first',
 'free',
 'binance',
 'now',
 'x200b',
 'vip',
 'big',
 'vibe',
 'recently',
 'limited',
 'invites',
 'group',
 'here',
 'me

In [102]:
conservative_ls = [
    '100k',
    'x200b',
    'binance',
    'invite',
    'vip',
    'coin',
    'pumping',
    '12d1kjww42',
    'pumps',
    'signals',
    '90k',
    'huuuuuge',
    '90000',
    '100000x',
    '10000x',
    'megapumpgroup',
    'returns',
    'chance',
    'amp',
    'group'    
]